## Proposed features for weights and biases
Draft Rank = 35% <br>
Projected Season Points = 25% <br>
Positions needing to be filled on the team = 15% <br>
Position Value = 10% <br>
Drafter's Pro Team Preferences = 7% <br>
Player Injury Implications = 6% <br>
Star Power = 4% <br>

In [1]:
import json
import ast
import numpy as np
import pandas as pd
from sklearn import preprocessing

In [2]:
# pull in our data from the csv
data = pd.read_csv('analysis/data/draft-players-2023.csv')

In [3]:
# The first column is the original indexes which are not needed, so we will drop it
data = data.drop(data.columns[0], axis=1)
data.head(5)

,draftRank,eligibleSlots,playerId,name,projPoints
0,1,"[11, 12, 13, 14, 16, 17]",39832,Shohei Ohtani,588.0
1,2,"[10, 5, 12, 16, 17]",36969,Juan Soto,602.0
2,3,"[13, 14, 16, 17]",32081,Gerrit Cole,629.0
3,4,"[3, 7, 19, 11, 12, 16, 17]",32801,Jose Ramirez,676.0
4,5,"[13, 14, 16, 17]",39878,Corbin Burnes,613.0


In [4]:
all_projected_points = data['projPoints'].sum()
print("The total projected points ESPN has calculated for all players in the 2023 season is:", f'{int(all_projected_points):,}')

The total projected points ESPN has calculated for all players in the 2023 season is: 150,474


## Position analysis
For each position that will be used in our league, we are going to evaluate the strength and value of players in those positions.
The positions our league uses is: Catcher, 1B, 2B, 3B, SS, 3 OF, DH, 4 Utility, and 9 Pitchers
SP are limited to 8 starts per week, so it is best to plan to have 6 SP and 3 RP

We are going to aggregate the following statistics:
- **Position Weight** = Projected fantasy score / Total fantasy point projections
- **Normalized Position Weight** = Position weight on a normalized scale
- **Max Projected Points** = Maximum amount of projected fantasy points for a player in this position
- **Min Projected Points** = The amount of fantasy points the lowest scoring drafted player would be projected to earn.  This is calculated as n = (# of starters for position * # of teams in league).  Ascending sorted projected fantasy players\[n]



In [5]:
# create an object of each position and their ID using the positions lookup file
position_map = {}
with open('lookups/espn-positions-map.json') as f:
  position_map = json.load(f)

In [6]:
# We will manually configure which IDs are being used by our league for positions
# and how many starters we want to have at each position to filter our positions object
starter_position_ids = [0, 1, 2, 3, 4, 5, 11, 14, 15]
position_starters_config = {"c": 1, "1b": 1, "2b": 1, "3b": 1, "ss": 1, "of": 3, "dh": 1, "sp": 6, "rp": 3, "UTIL": 4, "BENCH": 5}

position_map = {k: v for k, v in position_map.items() if v['posId'] in starter_position_ids}
# Note: we just removed UTIL and BENCH from the position map, even though they exist in the starters_config dict
# This is appropriate for our up front calculations, but we will still consider this data later

# appending the starters allowed to each position
for k, v in position_map.items():
  position_map[k]['startersAllowed'] = position_starters_config[str(v['abbrev']).lower()]

In [7]:
# We are going to do some analysis on each positon and update it to the position_map object
for position in starter_position_ids:

  # subset all player data based on who is eligible for the position sorted by projected points
  position_eligible_players = [values for index, values in data.iterrows() if position in ast.literal_eval(values['eligibleSlots'])]
  position_eligible_players = sorted(position_eligible_players, key=lambda k: k['projPoints'], reverse=True)

  # parse the max projected points, min projected points, and total projected points for the position
  max_projected_points = position_eligible_players[0]['projPoints']
  min_projected_points = position_eligible_players[(position_map[str(position)]['startersAllowed']*12)+1]['projPoints']
  sum_projected_points = sum([float(player['projPoints']) for player in position_eligible_players])

  # save each data point to the position_map object
  position_map[str(position)]['totalProjectedPoints'] = sum_projected_points
  position_map[str(position)]['maxProjectedPoints'] = max_projected_points
  position_map[str(position)]['minProjectedPoints'] = min_projected_points

In [8]:
# Let's take a look at an updated entry from the position_map object
position_map['0']

{'posId': 0,
 'name': 'Catcher',
 'abbrev': 'C',
 'startersAllowed': 1,
 'totalProjectedPoints': 10139.5,
 'maxProjectedPoints': 452.5,
 'minProjectedPoints': 316.5}

In [9]:
# We are going to calculate how many points each position is projected to contribute
# to overall scoring this year and turn that into a weighted score
for k, v in position_map.items():
  v['overallWeight'] = v['totalProjectedPoints'] / all_projected_points

In [10]:
# In order to do accurate analysis, we are also going to observe the normalized weight for each position
pos_points_list = [v['totalProjectedPoints'] for k, v in position_map.items()]
norm_pos_points_list = preprocessing.normalize([pos_points_list])[0]
position_map = [{**v, **{'normalizedPositionValue': norm_pos_points_list[i]}} for i, (k, v) in enumerate(position_map.items())]

In [11]:
print("Total Points:", pos_points_list)
print("Normalized Points:", norm_pos_points_list)
position_map[0]

Total Points: [10139.5, 15608.0, 15170.5, 16141.0, 13826.5, 35857.5, 22173.5, 43800.0, 18474.5]
Normalized Points: [0.14246494 0.21930005 0.21315296 0.22678896 0.1942691  0.50381544
 0.31154854 0.61541146 0.25957578]


{'posId': 0,
 'name': 'Catcher',
 'abbrev': 'C',
 'startersAllowed': 1,
 'totalProjectedPoints': 10139.5,
 'maxProjectedPoints': 452.5,
 'minProjectedPoints': 316.5,
 'overallWeight': 0.0673837340670149,
 'normalizedPositionValue': 0.14246494211383717}

## Evaluate Players:
We are going to weight the following features for individual players:
- What is the rank of the player compared to all projected points in their position
- Handle multiple positions through an averaging algorithm
- What is the normalized |weight of a players projected fantasy points
- How much of a negative impact to other teams does drafting the player create?

In [12]:
# loop through each player in our dataset and calculate their positional value
for index, player in data.iterrows():
  # identify the relevant starter positions for this player
  players_positions = [position for position in ast.literal_eval(player['eligibleSlots']) if position in starter_position_ids]

  # if there are multiple eligible positions, this will average them together
  scores = []
  for pos in players_positions:
    position_stats = [position for position in position_map if position['posId'] == pos]
    score = position_stats[0]['normalizedPositionValue'] * player['projPoints']
    scores.append(score)
  avg_score = np.mean(scores)

  # update the player's positional value in our original dataset
  data.loc[index, 'overallPositionalValue'] = avg_score

data.head()

,draftRank,eligibleSlots,playerId,name,projPoints,overallPositionalValue
0,1,"[11, 12, 13, 14, 16, 17]",39832,Shohei Ohtani,588.0,272.526238
1,2,"[10, 5, 12, 16, 17]",36969,Juan Soto,602.0,303.296895
2,3,"[13, 14, 16, 17]",32081,Gerrit Cole,629.0,387.093806
3,4,"[3, 7, 19, 11, 12, 16, 17]",32801,Jose Ramirez,676.0,181.958073
4,5,"[13, 14, 16, 17]",39878,Corbin Burnes,613.0,377.247223


## Determine Position Needs
- We will need to consider what fielding positions we still need on our team
- This should consider what positions we've already picked
- This should consider how much value is still left on the draft board for a given position at this moment in time
- This should consider how much value will still be left on the board by time we get to the next pick
* A super feature would be to predict, out of the most probable remaining draft pick outcomes from the current pick on, by all teams, which player at what pick delivers me the most value

In [13]:
# First, we are going to define the draft order for this season
draft_order_list = []

# We will use the draft-order.json file which describes the order that Fantasy teams will pick in the draft
with open('analysis/data/draft-order-2023.json', 'r') as f:
  draft_order = json.load(f)
  draft_order = sorted(draft_order, key=lambda k: k['draftOrder'])
  print("The draft order is:", [player['teamName'] for player in draft_order])
  rounds = 27

  # We will loop through the draft order and add the team name to the list for each pick
  # There will be some conditions applied to account for the "snake" draft order pattern
  for i in range(rounds):
    reverse_toggle = i % 2

    if i < 3:
      reverse_toggle = 1

    if reverse_toggle == 1:
      for player in draft_order:
        draft_order_list.append(player['teamName'])
    elif reverse_toggle == 0:
      for player in draft_order[::-1]:
        draft_order_list.append(player['teamName'])

print("There will be", len(draft_order_list), "picks over", int(len(draft_order_list) / len(draft_order)), "rounds")

The draft order is: ['BG 420', 'The Dissenters', "Loser's Club L", 'RIP Paulie Walnuts', 'Nomar Losing', 'Pecan Sandies', 'Rocket City Trash Pandas', '2-Time Champ Booch', 'Commissioner Getbot420', 'Big Trains', 'Hot Italian Snausages', 'Vandelay Industries']
There will be 324 picks over 27 rounds


Formula for calculating position needs:
$$
  \int \frac{StartersNeeded - \frac{PlayersOwnedForPosition}{PlayersEligiblePositions}}{BatterRosterSize -(PickNumber - 1)}
$$

In [ ]:
def determinePositionNeeds(picker, pick_number, current_pick_list, remaining_picks, full_draft_board):
  """
  This function will generate a pick for the given picker
  Parameters:
    picker (str): The name of the team that is picking
    pick_number (int): The number of the pick in the draft
    current_pick_list (list): A list of dictionaries that contain the picks for each team
    remaining_picks (list): A list of the remaining picks in the draft
    full_draft_board (DataFrame): A DataFrame that contains the full draft board
  """
  PICK_ROUND = np.ceil(pick_number / 12)
  BATTER_ELIGIBLE_POSITIONS = [0, 1, 2, 3, 4, 5, 11, 12]
  PITCHER_ELIGIBLE_POSITIONS = [14, 15]
  
  owners_current_picks = [pick['picks'] for pick in current_pick_list if pick['teamName'] == picker]
  pos_scores = {"c": 1, "1b": 1, "2b": 1, "3b": 1, "ss": 1, "of": 1, "dh": 1, "sp": 1, "rp": 1}
  for pick in owners_current_picks:
    player = full_draft_board.loc[full_draft_board['playerId'] == pick[0]['playerId']]
    num_eligible_positions = sum(player[['c', '1b', '2b', '3b', 'ss', 'of', 'dh', 'sp', 'rp']].values[0])
    players_pos = [index for index, pos in player[['c', '1b', '2b', '3b', 'ss', 'of', 'dh', 'sp', 'rp']].items() if pos == True]
    for pos in players_pos:
      if pos in ['c', '1b', '2b', '3b', 'ss', 'of', 'dh']:
        pos_scores[pos] -= position_starters_config.values() - (1 / sum([v['positions'] for k, v in current_pick_list.items() if v['positions'] in BATTER_ELIGIBLE_POSITIONS])) / (sum({k: v for k, v in position_starters_config.items() if k in ['c', '1b', '2b', '3b', 'ss', 'of', 'dh']}.values()) - (PICK_ROUND - 1))
      elif pos in ['sp', 'rp']:
        pos_scores[pos] -= lambda x: sum({k: v for k, v in position_starters_config.items() if k in [pos, 'BENCH']}.values()) / sum({k: v for k, v in position_starters_config.items() if k in ['sp', 'rp', 'BENCH']}.values())

In [ ]:
predicted_draft_picks = [{'teamName': team['teamName'], 'picks': []} for team in draft_order]

In [ ]:
def generatePick():
  return

In [ ]:
pick_number = 1
for picker in draft_order_list:
  for team in predicted_draft_picks:
    if team['teamName'] == picker:
      pick_index = generatePick(picker, pick_number, predicted_draft_picks, posEligData, data)
      team['picks'].append(posEligData.iloc[pick_index][['playerId', 'name']].to_dict())
      posEligData = posEligData.drop(pick_index)
      pick_number += 1
      team['picks'] = sorted(team['picks'], key=lambda k: k['playerId'])


In [ ]:
formula = sum({k: v for k, v in position_starters_config.items() if k in batter_pos}.values()) - sum({pick['name'] for pick in current_pick_list}.values())